<h1><p align="center"> 19<sup>th</sup> August'23 SQL Assignment </p></h1>

## Data Setup

In [1]:
import sqlite3
import pandas as pd
import zipfile
from tqdm import tqdm
import os

### Unzipping Data

In [2]:
zip_file_path = 'database/simplemaps_worldcities_basicv1.77.zip'
extract_to_dir = 'database/raw_data'

os.makedirs(extract_to_dir, exist_ok=True)

try:
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to_dir)
    print(f'Files extracted to {extract_to_dir}')
except zipfile.BadZipFile:
    print("Error: The ZIP file is corrupted or invalid.")
except FileNotFoundError:
    print("Error: The ZIP file was not found.")
except RuntimeError as e:
    print(f"Error: {e}")

Files extracted to database/raw_data


In [3]:
df = pd.read_csv("database/raw_data/worldcities.csv", nrows=10)
df.head()

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
0,Tokyo,Tokyo,35.6897,139.6922,Japan,JP,JPN,Tōkyō,primary,37732000,1392685764
1,Jakarta,Jakarta,-6.1750,106.8275,Indonesia,ID,IDN,Jakarta,primary,33756000,1360771077
2,Delhi,Delhi,28.6100,77.2300,India,IN,IND,Delhi,admin,32226000,1356872604
3,Guangzhou,Guangzhou,23.1300,113.2600,China,CN,CHN,Guangdong,admin,26940000,1156237133
4,Mumbai,Mumbai,19.0761,72.8775,India,IN,IND,Mahārāshtra,admin,24973000,1356226629


In [4]:
df.columns = ["DISTRICT", "NAME", "LAT_N", "LONG_W", "COUNTRY", "COUNTYCODE2", "COUNTYCODE", "STATE", "CAPITAL", "POPULATION", "ID"]

In [5]:
df.head()

,DISTRICT,NAME,LAT_N,LONG_W,COUNTRY,COUNTYCODE2,COUNTYCODE,STATE,CAPITAL,POPULATION,ID
0,Tokyo,Tokyo,35.6897,139.6922,Japan,JP,JPN,Tōkyō,primary,37732000,1392685764
1,Jakarta,Jakarta,-6.1750,106.8275,Indonesia,ID,IDN,Jakarta,primary,33756000,1360771077
2,Delhi,Delhi,28.6100,77.2300,India,IN,IND,Delhi,admin,32226000,1356872604
3,Guangzhou,Guangzhou,23.1300,113.2600,China,CN,CHN,Guangdong,admin,26940000,1156237133
4,Mumbai,Mumbai,19.0761,72.8775,India,IN,IND,Mahārāshtra,admin,24973000,1356226629


In [64]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   DISTRICT     10 non-null     object 
 1   NAME         10 non-null     object 
 2   LAT_N        10 non-null     float64
 3   LONG_W       10 non-null     float64
 4   COUNTRY      10 non-null     object 
 5   COUNTYCODE2  10 non-null     object 
 6   COUNTYCODE   10 non-null     object 
 7   STATE        10 non-null     object 
 8   CAPITAL      10 non-null     object 
 9   POPULATION   10 non-null     int64  
 10  ID           10 non-null     int64  
dtypes: float64(2), int64(2), object(7)
memory usage: 1012.0+ bytes


## Create Tabel And Insert Data into Table

In [6]:
# Connect to database
conn = sqlite3.connect('database/CITY_DATABASE.db')
c = conn.cursor()

# Create table
c.execute('''CREATE TABLE IF NOT EXISTS CITY
             (DISTRICT TEXT,
              NAME TEXT,
              LAT_N REAL,
              LONG_W REAL,
              COUNTRY TEXT,
              COUNTYCODE2 TEXT,
              COUNTYCODE TEXT,
              STATE TEXT,
              CAPITAL TEXT,
              POPULATION INTEGER,
              ID INTEGER PRIMARY KEY);''')

for idx in tqdm(range(df.shape[0]), desc="Inserting data"):
    insert_data_query = f'''INSERT INTO CITY 
            (DISTRICT, NAME, LAT_N, LONG_W, COUNTRY, COUNTYCODE2, COUNTYCODE, STATE, CAPITAL, POPULATION, ID)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?); '''
    data = (
        df.iloc[idx]["DISTRICT"],
        df.iloc[idx]["NAME"],
        df.iloc[idx]["LAT_N"],
        df.iloc[idx]["LONG_W"],
        df.iloc[idx]["COUNTRY"],
        df.iloc[idx]["COUNTYCODE2"],
        df.iloc[idx]["COUNTYCODE"],
        df.iloc[idx]["STATE"],
        df.iloc[idx]["CAPITAL"],
        df.iloc[idx]["POPULATION"],
        df.iloc[idx]["ID"],
        )
    # Replace NaN values with None
    data = [None if pd.isna(value) else value for value in data]
    # print(data)
    # Execute the command
    c.execute(insert_data_query, data)
    
# Commit the changes
conn.commit()

# Close connection
conn.close()

Inserting data:   0%|          | 0/10 [00:00<?, ?it/s]


IntegrityError: datatype mismatch

In [7]:
data

['Tokyo',
 'Tokyo',
 35.6897,
 139.6922,
 'Japan',
 'JP',
 'JPN',
 'Tōkyō',
 'primary',
 37732000,
 1392685764]

## SQL Questions

**Q1.** Query all columns for all American cities in the CITY table with populations larger than 100000. The CountryCode for America is USA.
The CITY table is described as follows:

CITY

|**Field**|**Type**|
|---|---|
| ID | NUMBER |
| NAME | VARCHAR2(17) |
| COUNTRYCODE | VARCHAR2(3) |
| DISTRICT | VARCHAR2(20) |
| POPULATION | NUMBER |


**Q2.** Query the NAME field for all American cities in the CITY table with populations larger than 120000. The CountryCode for America is USA.
The CITY table is described as follows:

CITY

|**Field**|**Type**|
|---|---|
| ID | NUMBER |
| NAME | VARCHAR2(17) |
| COUNTRYCODE | VARCHAR2(3) |
| DISTRICT | VARCHAR2(20) |
| POPULATION | NUMBER |

**Q3.** Query all columns (attributes) for every row in the CITY table. The CITY table is described as follows:

CITY

|**Field**|**Type**|
|---|---|
| ID | NUMBER |
| NAME | VARCHAR2(17) |
| COUNTRYCODE | VARCHAR2(3) |
| DISTRICT | VARCHAR2(20) |
| POPULATION | NUMBER |

**Q4.** Query all columns for a city in CITY with the ID 1661. The CITY table is described as follows:


CITY

|**Field**|**Type**|
|---|---|
| ID | NUMBER |
| NAME | VARCHAR2(17) |
| COUNTRYCODE | VARCHAR2(3) |
| DISTRICT | VARCHAR2(20) |
| POPULATION | NUMBER |

**Q5.** Query all attributes of every Japanese city in the CITY table. The COUNTRYCODE for Japan is JPN.
The CITY table is described as follows:

CITY

|**Field**|**Type**|
|---|---|
| ID | NUMBER |
| NAME | VARCHAR2(17) |
| COUNTRYCODE | VARCHAR2(3) |
| DISTRICT | VARCHAR2(20) |
| POPULATION | NUMBER |

**Q6.** Query the names of all the Japanese cities in the CITY table. The COUNTRYCODE for Japan is JPN.
The CITY table is described as follows:

CITY

|**Field**|**Type**|
|---|---|
| ID | NUMBER |
| NAME | VARCHAR2(17) |
| COUNTRYCODE | VARCHAR2(3) |
| DISTRICT | VARCHAR2(20) |
| POPULATION | NUMBER |

**Q7.** Query a list of CITY and STATE from the STATION table. The STATION table is described as follows:

STATION
|**Field**|**Type**|
|---|---|
|ID| NUMBER|
|CITY|VARCHAR2(21)|
|STATE|VARCHAR2(2)|
|LAT_N|NUMBER|
|LONG_W|NUMBER|

where LAT_N is the northern latitude and LONG_W is the western longitude.

**Q8.** Query a list of CITY names from STATION for cities that have an even ID number. Print the results in any order, but exclude duplicates from the answer.
The STATION table is described as follows:

STATION
|**Field**|**Type**|
|---|---|
|ID| NUMBER|
|CITY|VARCHAR2(21)|
|STATE|VARCHAR2(2)|
|LAT_N|NUMBER|
|LONG_W|NUMBER|

where LAT_N is the northern latitude and LONG_W is the western longitude.

**Q9.** Find the difference between the total number of CITY entries in the table and the number of distinct CITY entries in the table.
The STATION table is described as follows:

STATION
|**Field**|**Type**|
|---|---|
|ID| NUMBER|
|CITY|VARCHAR2(21)|
|STATE|VARCHAR2(2)|
|LAT_N|NUMBER|
|LONG_W|NUMBER|

where LAT_N is the northern latitude and LONG_W is the western longitude.

For example, if there are three records in the table with CITY values 'New York', 'New York', 'Bengalaru', there are 2 different city names: 'New York' and 'Bengalaru'. The query returns , because total number of records - number of unique city names = 3-2 =1

**Q10.** Query the two cities in STATION with the shortest and longest CITY names, as well as their respective lengths (i.e.: number of characters in the name). If there is more than one smallest or largest city, choose the one that comes first when ordered alphabetically.
The STATION table is described as follows:

STATION
|**Field**|**Type**|
|---|---|
|ID| NUMBER|
|CITY|VARCHAR2(21)|
|STATE|VARCHAR2(2)|
|LAT_N|NUMBER|
|LONG_W|NUMBER|

where LAT_N is the northern latitude and LONG_W is the western longitude.

Sample Input

For example, CITY has four entries: DEF, ABC, PQRS and WXY.
Sample Output

ABC 3 

PQRS 4

**Hint** -

When ordered alphabetically, the CITY names are listed as ABC, DEF, PQRS, and WXY, with lengths and. The longest name is PQRS, but there are options for shortest named city. Choose ABC, because it comes first alphabetically.
Note

You can write two separate queries to get the desired output. It need not be a single query

**Q11.** Query the list of CITY names starting with vowels (i.e., a, e, i, o, or u) from STATION. Your result cannot contain duplicates.

Input Format

The STATION table is described as follows:

STATION
|**Field**|**Type**|
|---|---|
|ID| NUMBER|
|CITY|VARCHAR2(21)|
|STATE|VARCHAR2(2)|
|LAT_N|NUMBER|
|LONG_W|NUMBER|

where LAT_N is the northern latitude and LONG_W is the western longitude.

**Q12.** Query the list of CITY names ending with vowels (a, e, i, o, u) from STATION. Your result cannot contain duplicates.
Input Format
The STATION table is described as follows:

STATION
|**Field**|**Type**|
|---|---|
|ID| NUMBER|
|CITY|VARCHAR2(21)|
|STATE|VARCHAR2(2)|
|LAT_N|NUMBER|
|LONG_W|NUMBER|

where LAT_N is the northern latitude and LONG_W is the western longitude.

**Q14.** Query the list of CITY names from STATION that do not end with vowels. Your result cannot contain duplicates.

Input Format
The STATION table is described as follows:

STATION
|**Field**|**Type**|
|---|---|
|ID| NUMBER|
|CITY|VARCHAR2(21)|
|STATE|VARCHAR2(2)|
|LAT_N|NUMBER|
|LONG_W|NUMBER|

where LAT_N is the northern latitude and LONG_W is the western longitude.

**Q15.** Query the list of CITY names from STATION that either do not start with vowels or do not end with vowels. Your result cannot contain duplicates.
Input Format
The STATION table is described as follows:

STATION
|**Field**|**Type**|
|---|---|
|ID| NUMBER|
|CITY|VARCHAR2(21)|
|STATE|VARCHAR2(2)|
|LAT_N|NUMBER|
|LONG_W|NUMBER|

where LAT_N is the northern latitude and LONG_W is the western longitude.

**Q16.** Query the list of CITY names from STATION that do not start with vowels and do not end with vowels. Your result cannot contain duplicates.

Input Format
The STATION table is described as follows:

STATION
|**Field**|**Type**|
|---|---|
|ID| NUMBER|
|CITY|VARCHAR2(21)|
|STATE|VARCHAR2(2)|
|LAT_N|NUMBER|
|LONG_W|NUMBER|

where LAT_N is the northern latitude and LONG_W is the western longitude.

**Q17.**

Table: Product

|**Column Name**|**Type**|
|---|---|
|product_id|int|
|product_name|varchar|
|unit_price|int|

product_id is the primary key of this table.
Each row of this table indicates the name and the price of each product.

Table: Sales

|**Column Name**|**Type**|
|---|---|
|seller_id| int|
|product_id|int|
|buyer_id| int|
|sale_date| date |
|quantity| int |
|price| int |

This table has no primary key, it can have repeated rows.

product_id is a foreign key to the Product table.
Each row of this table contains some information about one sale.

Write an SQL query that reports the products that were only sold in the first quarter of 2019. That is, between 2019-01-01 and 2019-03-31 inclusive.
Return the result table in any order.
The query result format is in the following example.

Input: 

Product table:

|**product_id**|**product_name**|**unit_price**|
|---|---|---|
|1|S8|1000|
|2|G4|800|
|3|iPhone|1400|

Sales table:

|**seller_id**|**product_id**|**buyer_id**|**sale_date**|**quantity**|**price**|
|---|---|---|---|---|---|
|1|1|1|2019-01-21|2|2000|
|1|2|2|2019-02-17|1|800|
|2|2|3|2019-06-02|1|800|
|3|3|4|2019-05-13|2|2800|

Output:

|**product_id**|**product_name**|
|---|---|
|1|S8|

**Explanation:**

The product with id 1 was only sold in the spring of 2019.

The product with id 2 was sold in the spring of 2019 but was also sold after the spring of 2019. 

The product with id 3 was sold after spring 2019.

We return only product 1 as it is the product that was only sold in the spring of 2019.

**Q18.**

Table: Views

|**Column Name**|**Type**|
|---|---|
|article_id|int|
|author_id|int|
|viewer_id|int|
|view_date|date|

There is no primary key for this table, it may have duplicate rows.

Each row of this table indicates that some viewer viewed an article (written by some author) on some date.

Note that equal author_id and viewer_id indicate the same person.


Write an SQL query to find all the authors that viewed at least one of their own articles. Return the result table sorted by id in ascending order.
The query result format is in the following example.


Input: 

Views table:

|**article_id**|**author_id**|**viewer_id**|**view_date**|
|---|---|---|---|
|1|3|5|2019-08-01|
|1|3|6|2019-08-02|
|2|7|7|2019-08-01|
|2|7|6|2019-08-02|
|4|7|1|2019-07-22|
|3|4|4|2019-07-21|
|3|4|4|2019-07-21|

Output:

|id|
|---|
|4|
|7|

**Q19.**

Table: Delivery


|**Column Name**|**Type**|
|---|---|
|delivery_id|int|
|customer_id|int|
|order_date|date|
|customer_pref_delivery_date|date|

delivery_id is the primary key of this table.
The table holds information about food delivery to customers that make orders at some date and specify a preferred delivery date (on the same order date or after it).

If the customer's preferred delivery date is the same as the order date, then the order is called immediately; otherwise, it is called scheduled.

Write an SQL query to find the percentage of immediate orders in the table, rounded to 2 decimal places.

The query result format is in the following example.

Input: 

Delivery table:

|**delivery_id**|**customer_id**|**order_date**|**customer_pref_ delivery_date**|
|---|---|---|---|
|1|1|2019-08-01|2019-08-02|
|2|5|2019-08-02|2019-08-02|
|3|1|2019-08-11|2019-08-11|
|4|3|2019-08-24|2019-08-26|
|5|4|2019-08-21|2019-08-22|
|6|2|2019-08-11|2019-08-13|

Output:

|**immediate_percentage**|
|---|
|33.33|

Explanation: The orders with delivery id 2 and 3 are immediate while the others are scheduled.

**Q20.**

Table: Ads

|**Column Name**|**Type**|
|---|---|
|ad_id|int|
|user_id|int|
|action|enum|


(ad_id, user_id) is the primary key for this table.
Each row of this table contains the ID of an Ad, the ID of a user, and the action taken by this user regarding this Ad.

The action column is an ENUM type of ('Clicked', 'Viewed', 'Ignored').

A company is running Ads and wants to calculate the performance of each Ad. 

Performance of the Ad is measured using Click-Through Rate (CTR) where:

```Python
# add equation here
```


Write an SQL query to find the ctr of each Ad. Round ctr to two decimal points.

Return the result table ordered by ctr in descending order and by ad_id in ascending order in case of a tie.

The query result format is in the following example.

Input: 
Ads table:

|**ad_id**|**user_id**|**action**|
|---|---|---|
|1|1|Clicked|
|2|2|Clicked|
|3|3|Viewed|
|5|5|Ignored|
|1|7|Ignored|
|2|7|Viewed|
|3|5|Clicked|
|1|4|Viewed|
|2|11|Viewed|
|1|2|Clicked|

Output:

|**ad_id**|**ctr**|
|---|---|
|1|66.67|
|3|50|
|2|33.33|
|5|0|

Explanation:

for ad_id = 1, ctr = (2/(2+1)) * 100 = 66.67

for ad_id = 2, ctr = (1/(1+2)) * 100 = 33.33

for ad_id = 3, ctr = (1/(1+1)) * 100 = 50.00

for ad_id = 5, ctr = 0.00, Note that ad_id = 5 has no clicks or views. 

Note that we do not care about Ignored Ads.


<i>"Thank you for exploring all the way to the end of my page!"</i>

<p>
regards, <br>
<a href="https:www.github.com/Rahul-404/">Rahul Shelke</a>
</p>